In [5]:
import pickle
import re
from sklearn.metrics import confusion_matrix
import requests, bs4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import os

In [98]:
db = pd.read_csv('WNBA18.csv')

In [23]:
team_abbrevs = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'L.A. Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'Las Vegas Aces': 'LVA'
}

In [100]:
def date_formatter(date):

    if len(str(date.month)) != 2:
        month = '0'+ str(date.month)
    else:
        month = str(date.month)
        
    if len(str(date.day)) != 2:
        day = '0' + str(date.day)
    else:
        day = str(date.day)
        
    newdate = str(date.year) + month + day
    return newdate

In [101]:
db['Date'] = pd.to_datetime(db['Date'])
db['Date'] = db['Date'].apply(date_formatter)

In [102]:
db['Date']

0      20180520
1      20180523
2      20180526
3      20180529
4      20180603
5      20180605
6      20180608
7      20180610
8      20180612
9      20180614
10     20180616
11     20180619
12     20180622
13     20180627
14     20180629
15     20180701
16     20180706
17     20180708
18     20180711
19     20180713
20     20180715
21     20180717
22     20180719
23     20180722
24     20180724
25     20180731
26     20180803
27     20180805
28     20180807
29     20180809
         ...   
376    20180531
377    20180601
378    20180603
379    20180608
380    20180610
381    20180612
382    20180613
383    20180615
384    20180617
385    20180619
386    20180622
387    20180624
388    20180627
389    20180629
390    20180701
391    20180705
392    20180707
393    20180710
394    20180713
395    20180715
396    20180719
397    20180722
398    20180801
399    20180805
400    20180807
401    20180809
402    20180811
403    20180815
404    20180817
405    20180819
Name: Date, Length: 406,

In [103]:
# Moneyline scrape

scraped_dates = []
moneydb = pd.DataFrame()

for date in db['Date']:
    
    #if date == 20160616:
     #   pass
    
    if date not in scraped_dates:
        print("Date: ", date)
        url = f'https://www.sportsbookreview.com/betting-odds/wnba-basketball/money-line/?date={date}'
        res = requests.get(url, 'html.parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'lxml')
        linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
        test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
        teams = [team_abbrevs[team.get_text()] for team in test_teams]
        test_lines = linesoup.find_all('span', {'class':'opener'})
        lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') and len(line.get_text()) > 1 or line.get_text().startswith('+') and len(line.get_text()) > 1]
        print("teams:",len(teams),"lines",len(lines))
        
        if len(teams) != len(lines):
            pass
        else:
            moneylines = {'Team':teams, 'ML Odds': lines, 'Date':[date]*len(teams)}
        
            MLdb = pd.DataFrame.from_dict(moneylines)
            moneydb = pd.concat([moneydb, MLdb])
            
        scraped_dates.append(date)
        
# moneydb.to_csv('MLodds.csv')

Date:  20180520
teams: 12 lines 12
Date:  20180523
teams: 6 lines 6
Date:  20180526
teams: 4 lines 4
Date:  20180529
teams: 6 lines 6
Date:  20180603
teams: 8 lines 8
Date:  20180605
teams: 4 lines 4
Date:  20180608
teams: 6 lines 6
Date:  20180610
teams: 8 lines 8
Date:  20180612
teams: 8 lines 8
Date:  20180614
teams: 2 lines 2
Date:  20180616
teams: 6 lines 6
Date:  20180619
teams: 10 lines 10
Date:  20180622
teams: 12 lines 12
Date:  20180627
teams: 6 lines 6
Date:  20180629
teams: 8 lines 8
Date:  20180701
teams: 10 lines 0
Date:  20180706
teams: 2 lines 2
Date:  20180708
teams: 6 lines 6
Date:  20180711
teams: 6 lines 6
Date:  20180713
teams: 8 lines 8
Date:  20180715
teams: 10 lines 10
Date:  20180717
teams: 4 lines 4
Date:  20180719
teams: 6 lines 6
Date:  20180722
teams: 8 lines 8
Date:  20180724
teams: 8 lines 8
Date:  20180731
teams: 6 lines 6
Date:  20180803
teams: 6 lines 6
Date:  20180805
teams: 8 lines 8
Date:  20180807
teams: 8 lines 8
Date:  20180809
teams: 6 lines 6
D

In [44]:
date = 20190524


url = f'https://www.sportsbookreview.com/betting-odds/wnba-basketball/money-line/?date={date}'
res = requests.get(url, 'html.parser')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'lxml')
linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
teams = [team_abbrevs[team.get_text()] for team in test_teams]
test_lines = linesoup.find_all('span', {'class':'opener'})
lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') 
         and len(line.get_text()) > 1 or line.get_text().startswith('+') and len(line.get_text()) > 1]
print("teams:",len(teams),"lines",len(lines))
moneylines = {'Team':teams, 'ML Odds': lines, 'Date':[date]*len(teams)}


teams: 4 lines 0


In [45]:
teams

['DAL', 'ATL', 'IND', 'NYL']

In [ ]:
moneydb = pd.concat([moneydb, MLdb])

In [104]:
moneydb = moneydb.reset_index(drop = True)

In [105]:
moneydb.to_csv('moneydb18.csv', index = False)

In [106]:
db['Odds'] = np.zeros(len(db['Team']))

In [107]:
for i, date in enumerate(db['Date']):
    
    if date in list(moneydb['Date']):
        
        db['Odds'][i] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds']
        print("Team:", db['Team'][i], "Odds:", moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds'])
        
    else:
        db['Odds'][i] = np.nan

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Team: ATL Odds: 4    +170
Name: ML Odds, dtype: object
Team: ATL Odds: 12    +125
Name: ML Odds, dtype: object
Team: ATL Odds: 19    -155
Name: ML Odds, dtype: object
Team: ATL Odds: 25    +260
Name: ML Odds, dtype: object
Team: ATL Odds: 31    -102
Name: ML Odds, dtype: object
Team: ATL Odds: 39    +340
Name: ML Odds, dtype: object
Team: ATL Odds: 40    -200
Name: ML Odds, dtype: object
Team: ATL Odds: 52    +280
Name: ML Odds, dtype: object
Team: ATL Odds: 60    +310
Name: ML Odds, dtype: object
Team: ATL Odds: 63    -550
Name: ML Odds, dtype: object
Team: ATL Odds: 64    -220
Name: ML Odds, dtype: object
Team: ATL Odds: 72    +230
Name: ML Odds, dtype: object
Team: ATL Odds: 81    +210
Name: ML Odds, dtype: object
Team: ATL Odds: 92    +105
Name: ML Odds, dtype: object
Team: ATL Odds: 102    +800
Name: ML Odds, dtype: object
Team: ATL Odds: 107    +178
Name: ML Odds, dtype: object
Team: ATL Odds: 109    +165
Name: ML Odds, dtype: object


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Team: ATL Odds: 116    +810
Name: ML Odds, dtype: object
Team: ATL Odds: 121    -525
Name: ML Odds, dtype: object
Team: ATL Odds: 131    -118
Name: ML Odds, dtype: object
Team: ATL Odds: 140    +285
Name: ML Odds, dtype: object
Team: ATL Odds: 143    -450
Name: ML Odds, dtype: object
Team: ATL Odds: 149    +103
Name: ML Odds, dtype: object
Team: ATL Odds: 162    +160
Name: ML Odds, dtype: object
Team: ATL Odds: 165    -155
Name: ML Odds, dtype: object
Team: ATL Odds: 171    -400
Name: ML Odds, dtype: object
Team: ATL Odds: 180    +205
Name: ML Odds, dtype: object
Team: ATL Odds: 185    -425
Name: ML Odds, dtype: object
Team: ATL Odds: 195    -118
Name: ML Odds, dtype: object
Team: ATL Odds: 199    -310
Name: ML Odds, dtype: object
Team: ATL Odds: 202    -210
Name: ML Odds, dtype: object
Team: ATL Odds: 220    +130
Name: ML Odds, dtype: object
Team: ATL Odds: 224    -175
Name: ML Odds, dtype: object
Team: CON Odds: 1    -1500
Name: ML Odds, dtype: object
Team: CON Odds: 237    +120
Name

In [108]:
db.to_csv('WNBA18.csv', index = False)

## Below is for troubleshooting

In [9]:
url = 'https://www.sportsbookreview.com/betting-odds/wnba-basketball/?date=20180703'

In [10]:
res = requests.get(url, 'html.parser')
soup = bs4.BeautifulSoup(res.text, 'lxml')
linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
games = soup.find('div', {'class':'_1eZfC'})

In [19]:
pd.read_html(str(games), flavor = 'bs4', attrs = {'class':'_1eZfC'})

ValueError: No tables found

### Scraping lines remnants from NBA scraper

In [289]:
# Point spread scrape
scraped_dates = []
spreaddb = pd.DataFrame()

for date in db['Date']:
    if date not in scraped_dates:
        print("Date: ", date)
        url = f'https://www.sportsbookreview.com/betting-odds/nba-basketball/pointspread/?date={date}'
        res = requests.get(url, 'html.parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'lxml')
        linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
        test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
        teams = [team_abbrevs[team.get_text()] for team in test_teams]
        test_lines = linesoup.find_all('span', {'class':'_3Nv_7 opener'})
        lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') or line.get_text().startswith('+') or line.get_text().startswith('P')]
        print("teams:",len(teams),"lines",len(lines))
        spread = {'Team':teams, 'Spread': lines, 'Date':[date]*len(lines)}
        spreads = pd.DataFrame.from_dict(spread)
        spreaddb = pd.concat([spreaddb, spreads])
        scraped_dates.append(date)

spreaddb.to_csv('Spreads.csv', index = False)

Date:  20181017
teams: 22 lines 22
Date:  20181019
teams: 18 lines 18
Date:  20181021
teams: 8 lines 8
Date:  20181024
teams: 22 lines 22
Date:  20181025
teams: 8 lines 8
Date:  20181027
teams: 18 lines 18
Date:  20181030
teams: 16 lines 16
Date:  20181101
teams: 12 lines 12
Date:  20181103
teams: 16 lines 16
Date:  20181105
teams: 18 lines 18
Date:  20181107
teams: 20 lines 20
Date:  20181110
teams: 20 lines 20
Date:  20181113
teams: 6 lines 6
Date:  20181114
teams: 22 lines 22
Date:  20181119
teams: 18 lines 18
Date:  20181121
teams: 26 lines 26
Date:  20181123
teams: 28 lines 28
Date:  20181124
teams: 14 lines 14
Date:  20181126
teams: 14 lines 14
Date:  20181128
teams: 20 lines 20
Date:  20181130
teams: 22 lines 22
Date:  20181201
teams: 14 lines 14
Date:  20181203
teams: 14 lines 14
Date:  20181205
teams: 20 lines 20
Date:  20181207
teams: 20 lines 20
Date:  20181208
teams: 18 lines 18
Date:  20181210
teams: 22 lines 22
Date:  20181212
teams: 22 lines 22
Date:  20181214
teams: 18 

In [27]:
db['Odds'] = np.zeros(len(db['Team']))
db['Line'] = np.zeros(len(db['Team']))

In [297]:
date=db['Date'][0]
team = db['Team'][0]
print(date, team)

20181017 CLE


In [323]:
#db['Odds'][0] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][0])]['ML Odds']
#db['Line'][0] = spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][0])]['Spread']

moneydb.reset_index(drop = True, inplace = True)
spreaddb.reset_index(drop = True, inplace = True)



In [18]:
print(re.sub("½", ".5", spreaddb['Spread'][0]))

for i, spread in enumerate(spreaddb['Spread']):
    spreaddb['Spread'][i] = re.sub("½", ".5", spread)
    print(spreaddb['Spread'][i])

+7
+7
-7
-1.5
+1.5
-2
+2


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


+4.5
-4.5
+3.5
-3.5
+11.5
-11.5
+7.5
-7.5
+2
-2
-7
+7
PK
PK
PK
PK
-2
+2
+2.5
-2.5
+11
-11
+3
-3
+8.5
-8.5
+7.5
-7.5
+3.5
-3.5
-2
+2
+1.5
-1.5
+6.5
-6.5
+9
-9
-3.5
+3.5
-3
+3
+3
-3
-1.5
+1.5
+8
-8
+8
-8
-3
+3
+2.5
-2.5
+2.5
-2.5
+4.5
-4.5
-2.5
+2.5
-3
+3
+10
-10
+7
-7
-3
+3
+1.5
-1.5
-4
+4
-3
+3
-1
+1
+6
-6
+5
-5
-6
+6
+1.5
-1.5
+9
-9
+1.5
-1.5
+9
-9
+4
-4
+4.5
-4.5
+4.5
-4.5
+9
-9
+6.5
-6.5
+2.5
-2.5
+2.5
-2.5
+2
-2
+1.5
-1.5
+4.5
-4.5
-6
+6
-2
+2
+3
-3
+3.5
-3.5
+7.5
-7.5
-1.5
+1.5
+9.5
-9.5
-5.5
+5.5
-8
+8
+3.5
-3.5
+4
-4
+3.5
-3.5
PK
PK
+4.5
-4.5
+3.5
-3.5
+2.5
-2.5
+4.5
-4.5
+2
-2
-1
+1
+14
-14
+5
-5
-1.5
+1.5
-6.5
+6.5
PK
PK
+1.5
-1.5
-3.5
+3.5
+10
-10
+2.5
-2.5
+10
-10
-8.5
+8.5
+5
-5
+13.5
-13.5
-3
+3
+9
-9
+5.5
-5.5
+2.5
-2.5
+2
-2
+8.5
-8.5
-1
+1
-1
+1
-5.5
+5.5
-6.5
+6.5
+3.5
-3.5
+13.5
-13.5
-6
+6
+12
-12
+12
-12
+10
-10
-2
+2
+8.5
-8.5
+1.5
-1.5
+9.5
-9.5
-4.5
+4.5
-5
+5
+3
-3
-1.5
+1.5
-1.5
+1.5
+12.5
-12.5
+10
-10
-7
+7
+5
-5
+7.5
-7.5
+3
-3
-3
+3
+3.5
-3.5
+3.5
-3.5
-7.5

In [50]:
spreaddb['Spread'] = spreaddb['Spread'].replace('PK',0)
spreaddb['Spread'] = spreaddb['Spread'].apply(lambda x: float(x))

In [51]:
#spreaddb['Date'].apply(lambda x: int(x))
#moneydb['Date'].apply(lambda x: int(x))
moneydb['ML Odds'] = moneydb['ML Odds'].apply(lambda x: int(x))

In [52]:
for i, date in enumerate(db['Date']):
    
    db['Odds'][i] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds']
    print("Team:", db['Team'][i], "Odds:", moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds'])
    db['Line'][i] = spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])]['Spread']
    print("Team:", db['Team'][i], "Spread:", spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])]['Spread'])

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Team: CLE Odds: 10    600
Name: ML Odds, dtype: int64
Team: CLE Spread: 10    11.5
Name: Spread, dtype: float64
Team: CLE Odds: 30    305
Name: ML Odds, dtype: int64
Team: CLE Spread: 30    8.5
Name: Spread, dtype: float64
Team: CLE Odds: 41   -260
Name: ML Odds, dtype: int64
Team: CLE Spread: 41   -6.5
Name: Spread, dtype: float64
Team: CLE Odds: 49   -150
Name: ML Odds, dtype: int64
Team: CLE Spread: 49   -3.0
Name: Spread, dtype: float64
Team: CLE Odds: 70    250
Name: ML Odds, dtype: int64
Team: CLE Spread: 70    7.0
Name: Spread, dtype: float64
Team: CLE Odds: 87    245
Name: ML Odds, dtype: int64
Team: CLE Spread: 87    6.0
Name: Spread, dtype: float64
Team: CLE Odds: 97   -170
Name: ML Odds, dtype: int64
Team: CLE Spread: 97   -4.0
Name: Spread, dtype: float64
Team: CLE Odds: 117    195
Name: ML Odds, dtype: int64
Team: CLE Spread: 117    6.0
Name: Spread, dtype: float64
Team: CLE Odds: 128    380
Name: ML Odds, dtype: int64
Team: CLE Spread: 128    9.5
Name: Spread, dtype: floa

In [53]:
spreaddb.to_csv('Spreads.csv')

In [54]:
moneydb.to_csv('MLodds.csv')

In [56]:
db.to_csv('2017_2019stats.csv')